Used to search for files that match a specified pattern.

In [1]:
!pip install pandas
import pandas as pd
import numpy as np
import glob

Xiao Hei Cleaning


In [2]:
# Load the qrels file
qrels_file_path = 'qrels.trec8.adhoc.csv'
qrels_df = pd.read_csv(qrels_file_path, sep=' ', header=None, names=['query_id', 'ignore', 'doc_id', 'relevance'])

#show qrels_dataframe
qrels_df.head()
print(qrels_df.columns)

Index(['query_id', 'ignore', 'doc_id', 'relevance'], dtype='object')


In [3]:
# Get a list of all CSV files in the input directory
input_files = glob.glob('input/*.csv')

# Create an empty list to store the input files dataframes
input_dfs = []

# Loop through each input CSV file
for filename in input_files:
    # Read the input CSV file into a dataframe
    df = pd.read_csv(filename, sep='\t', header=None)
    # Append the dataframe to the list of dataframes
    input_dfs.append(df)
    
raw_data = pd.concat(input_dfs, ignore_index=True)
print('Shape of raw_data:', raw_data.shape)

############################################## Data cleaning ##############################################

# Loop through each input dataframe and filter out rows where does not meet the clean up requirement
# 1. data_id  is not present in qrels_data_ids
# 2. relevant_score equal to 0
# 3. duplicate ranking

# Extract the valid values from the third column of qrels_df
qrels_data_ids = set(qrels_df.iloc[:, 2])

cleaned_dfs = []

for df in input_dfs:
    # Get rows where the data id is present in the qrels data ids
    df_with_present_data_id = df.iloc[:, 2].isin(qrels_data_ids)
    
    # Get rows where relevant_score does not equal 0
    df_with_relevant_score_not_zero = df.iloc[:, 4] != 0
    
    # Combine targeted rows for first and second rule
    cleaned_df = df[df_with_present_data_id & df_with_relevant_score_not_zero]

    # Get rows where ranking is not duplicated
    cleaned_df = cleaned_df.drop_duplicates(subset=[4])
    cleaned_dfs.append(cleaned_df)

# Print the shape of the cleaned data
print("this is input data:", cleaned_dfs)

Shape of raw_data: (689266, 6)
this is input data: [         0   1              2    3             4           5
0      401  Q0     FT924-5091    0  10015.175781  acsys8aln2
1      401  Q0    FBIS3-39240    1  10014.111328  acsys8aln2
2      401  Q0     FT924-4470    2  10013.638672  acsys8aln2
3      401  Q0    FBIS4-18182    3  10013.591797  acsys8aln2
4      401  Q0    FBIS3-59055    4  10013.441406  acsys8aln2
...    ...  ..            ...  ...           ...         ...
49910  450  Q0    FBIS3-12998  910  10012.927734  acsys8aln2
49922  450  Q0  LA100890-0074  922  10012.846680  acsys8aln2
49934  450  Q0    FBIS4-59198  934  10012.767578  acsys8aln2
49961  450  Q0    FBIS4-58516  961  10012.646484  acsys8aln2
49968  450  Q0    FBIS4-37789  968  10012.623047  acsys8aln2

[12367 rows x 6 columns],          0   1            2    3       4         5
0      401  Q0   FT942-5121    2  8.3846  apl8c621
2      401  Q0   FT924-7558    3  6.6410  apl8c621
3      401  Q0  FT931-10513    4  5.

In [4]:
# Define the column names
columns = ['query_id', 'ignore', 'doc_id', 'rank', 'score', 'ignore2']

# Create an empty list to store the cleaned dataframes
cleaned_dfs_with_names = []

# Iterate over each cleaned dataframe in cleaned_dfs
for df in cleaned_dfs:
    # Rename the columns
    df = df.rename(columns=dict(enumerate(columns)))
    # Append the renamed dataframe to the new list
    cleaned_dfs_with_names.append(df)

In [5]:
print(cleaned_dfs_with_names)

[       query_id ignore         doc_id  rank         score     ignore2
0           401     Q0     FT924-5091     0  10015.175781  acsys8aln2
1           401     Q0    FBIS3-39240     1  10014.111328  acsys8aln2
2           401     Q0     FT924-4470     2  10013.638672  acsys8aln2
3           401     Q0    FBIS4-18182     3  10013.591797  acsys8aln2
4           401     Q0    FBIS3-59055     4  10013.441406  acsys8aln2
...         ...    ...            ...   ...           ...         ...
49910       450     Q0    FBIS3-12998   910  10012.927734  acsys8aln2
49922       450     Q0  LA100890-0074   922  10012.846680  acsys8aln2
49934       450     Q0    FBIS4-59198   934  10012.767578  acsys8aln2
49961       450     Q0    FBIS4-58516   961  10012.646484  acsys8aln2
49968       450     Q0    FBIS4-37789   968  10012.623047  acsys8aln2

[12367 rows x 6 columns],        query_id ignore       doc_id  rank   score   ignore2
0           401     Q0   FT942-5121     2  8.3846  apl8c621
2           

In [6]:
# loop over the dataframes and print the first and third columns
for i, cleaned_df in enumerate(cleaned_dfs_with_names):
    print(f"Columns of dataframe {i+1}:")
    print("First column:", cleaned_df.iloc[:, 0])
    print("Third column:", cleaned_df.iloc[:, 2])

Columns of dataframe 1:
First column: 0        401
1        401
2        401
3        401
4        401
        ... 
49910    450
49922    450
49934    450
49961    450
49968    450
Name: query_id, Length: 12367, dtype: int64
Third column: 0           FT924-5091
1          FBIS3-39240
2           FT924-4470
3          FBIS4-18182
4          FBIS3-59055
             ...      
49910      FBIS3-12998
49922    LA100890-0074
49934      FBIS4-59198
49961      FBIS4-58516
49968      FBIS4-37789
Name: doc_id, Length: 12367, dtype: object
Columns of dataframe 2:
First column: 0        401
2        401
3        401
4        401
5        401
        ... 
49864    450
49900    450
49901    450
49971    450
49989    450
Name: query_id, Length: 17558, dtype: int64
Third column: 0         FT942-5121
2         FT924-7558
3        FT931-10513
4         FT944-6909
5         FT924-4344
            ...     
49864    FBIS4-58518
49900    FBIS4-12893
49901     FT944-1732
49971     FT942-4898
49989    FT921-1

Data Analysis Part 

In [7]:
merged_df = []
for df in cleaned_dfs_with_names:
    cleaned_df_with_qrel = pd.merge(df, qrels_df, on=['query_id', 'doc_id'], how='left')
    merged_df.append(cleaned_df_with_qrel)

In [8]:
print(merged_df)

[       query_id ignore_x         doc_id  rank         score     ignore2  \
0           401       Q0     FT924-5091     0  10015.175781  acsys8aln2   
1           401       Q0    FBIS3-39240     1  10014.111328  acsys8aln2   
2           401       Q0     FT924-4470     2  10013.638672  acsys8aln2   
3           401       Q0    FBIS4-18182     3  10013.591797  acsys8aln2   
4           401       Q0    FBIS3-59055     4  10013.441406  acsys8aln2   
...         ...      ...            ...   ...           ...         ...   
12362       450       Q0    FBIS3-12998   910  10012.927734  acsys8aln2   
12363       450       Q0  LA100890-0074   922  10012.846680  acsys8aln2   
12364       450       Q0    FBIS4-59198   934  10012.767578  acsys8aln2   
12365       450       Q0    FBIS4-58516   961  10012.646484  acsys8aln2   
12366       450       Q0    FBIS4-37789   968  10012.623047  acsys8aln2   

       ignore_y  relevance  
0           0.0        1.0  
1           0.0        1.0  
2          

In [9]:
calculate_metrics_precisions = []
calculate_metrics_avg_precisions = []
for df in merged_df:
    # group by query_id
    grouped = df.groupby('query_id')

    # calculate precision for top 10
    precisions = grouped.apply(lambda x: np.sum(x['relevance'].head(10)) / 10)
    calculate_metrics_precisions.append(precisions)

    # calculate average precision
    avg_precisions = grouped.apply(lambda x: np.sum(x['relevance'] * np.cumsum(x['relevance']) / np.arange(1, len(x['relevance']) + 1)) / np.sum(x['relevance']))
    calculate_metrics_avg_precisions.append(avg_precisions)


C:\Users\acer\AppData\Local\Temp/ipykernel_10292/2659911141.py:12: RuntimeWarning: invalid value encountered in double_scalars
  avg_precisions = grouped.apply(lambda x: np.sum(x['relevance'] * np.cumsum(x['relevance']) / np.arange(1, len(x['relevance']) + 1)) / np.sum(x['relevance']))


In [10]:
print("calculate_metrics_precisions",calculate_metrics_precisions)
print("calculate_metrics_avg_pr",calculate_metrics_avg_precisions)

calculate_metrics_precisions [query_id
401    0.7
402    1.0
403    1.0
404    0.4
405    0.2
406    0.4
407    0.6
408    0.4
409    0.5
410    1.0
411    0.5
412    0.7
413    0.9
414    0.4
415    1.0
416    0.6
417    0.7
418    0.7
419    0.2
420    0.3
421    0.0
422    0.6
423    0.8
424    0.7
425    0.9
426    0.4
427    0.5
428    0.2
429    0.3
430    0.4
431    0.8
432    0.0
433    0.0
434    0.9
435    0.1
436    0.2
437    0.0
438    0.2
439    0.1
440    0.3
441    0.8
442    0.0
443    0.2
444    0.8
445    0.2
446    0.5
447    0.2
448    0.1
449    0.3
450    0.9
dtype: float64, query_id
401    0.8
402    0.9
403    0.9
404    0.4
405    0.6
406    0.6
407    0.9
408    0.2
409    0.3
410    1.0
411    0.9
412    0.0
413    0.7
414    0.3
415    1.0
416    0.4
417    0.5
418    0.7
419    0.3
420    0.8
421    0.0
422    0.4
423    0.9
424    0.5
425    0.7
426    0.6
427    0.5
428    0.5
429    0.1
430    0.3
431    1.0
432    0.0
433    0.1
434    0.5
435    0.4
4

In [11]:
# assuming the lists have the same length
results_df = pd.concat([pd.DataFrame(calculate_metrics_precisions).T],  axis=1)
results_df.columns = ["Precision@10_" + str(i) for i in range(15)] 

# calculate the overall precision@10
overall_precision = results_df.mean(axis=1)

# add the new column to the DataFrame
results_df['Overall Precision@10'] = overall_precision

# calculate the overall row precision@10
overall_precision_row = results_df.mean(axis=0)

# add the new row to the DataFrame
results_df.loc["Overall"] = overall_precision_row

# show the new table
print(results_df)
results_df.to_csv('resultsT10.csv')

          Precision@10_0  Precision@10_1  Precision@10_2  Precision@10_3  \
query_id                                                                   
401                0.700            0.80           0.400           0.000   
402                1.000            0.90           0.600           0.000   
403                1.000            0.90           0.900           0.100   
404                0.400            0.40           0.200           0.200   
405                0.200            0.60           0.300           0.000   
406                0.400            0.60           0.500           0.000   
407                0.600            0.90           0.900           0.400   
408                0.400            0.20           0.200           0.100   
409                0.500            0.30           0.100           0.000   
410                1.000            1.00           1.000           0.200   
411                0.500            0.90           0.600             NaN   
412         

In [12]:
results_df = pd.concat([pd.DataFrame(calculate_metrics_avg_precisions).T],  axis=1)
results_df.columns = ["MAP_" + str(i) for i in range(15)] 


# calculate the overall precision@10
overall_precision = results_df.mean(axis=1)

# add the new column to the DataFrame
results_df['Overall Precision@10'] = overall_precision

# calculate the overall row precision@10
overall_precision_row = results_df.mean(axis=0)

# add the new row to the DataFrame
results_df.loc["Overall"] = overall_precision_row

# show the new table
print(results_df)
results_df.to_csv('resultsMAP.csv')

             MAP_0     MAP_1     MAP_2     MAP_3     MAP_4     MAP_5  \
query_id                                                               
401       0.413923  0.542508  0.378983       NaN  0.059724  0.195670   
402       0.477646  0.433076  0.293881       NaN  0.273211  0.291548   
403       0.897210  0.827710  0.819372  0.133333  0.904492  0.863840   
404       0.276444  0.278495  0.242244  0.291667  0.254519  0.244822   
405       0.197862  0.359231  0.215272       NaN  0.335481  0.346934   
406       0.408839  0.454128  0.498380       NaN  0.451879  0.445387   
407       0.409770  0.570476  0.596186  0.887500  0.508897  0.526538   
408       0.424536  0.378143  0.290760  0.500000  0.409963  0.277905   
409       0.277595  0.233760  0.186078       NaN  0.177774  0.273157   
410       0.918637  0.834999  0.885252  0.583333  0.910298  0.884489   
411       0.581296  0.653910  0.340042       NaN  0.216702  0.290932   
412       0.372499  0.097680  0.568747       NaN  0.088415  0.49